In [1]:
# !pip install langdetect
# !pip install textblob
# !pip install  polyglot
# !pip install guess_language-spirit
# !pip install langid
# !pip install fasttext
# !pip install pycld3



In [2]:
words = ["Я люблю вкусные пампушки", "中文字符", "にほんご", "لعَرَبِيَّالعَرَبِيَّة", "Hello 中文文"]

In [3]:
from langdetect import detect
for word in words:
    lang = detect(word)
    print(lang)

ru
zh-cn
ja
ar
it


In [4]:
from textblob import TextBlob
lang = []
for word in words:
    detector = TextBlob(word)
    lang.append(detector.detect_language())
print(lang)
# mush provide a string at least 3 characters    

HTTPError: HTTP Error 429: Too Many Requests

In [ ]:
from polyglot.detect import Detector
# can detect mixed languages
lang = []
for word in words:
    detector = Detector(word)
    lang.append(detector.languages())
print(lang)
# need pyicu package
# https://polyglot.readthedocs.io/en/latest/Installation.html
# !pip install pycld3
# !pip install pycld2

In [5]:
import urllib
rawdata = urllib.request.urlopen('http://yahoo.co.jp/').read()
import chardet 
chardet.detect(rawdata)


{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}

In [6]:
import langid 
langid.classify("What's wrong with you? hello")

('en', -53.71452236175537)

In [23]:
# Get the classifier of FastText
# !wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.ftz
import fasttext
model = fasttext.load_model('lid.176.ftz')
lang = list(map(model.predict, words))
print(lang[0][0][-1])

__label__ru


In [8]:
import cld3
lang = list(map(cld3.get_language, words))
print(lang)


[LanguagePrediction(language='be', probability=0.9016516208648682, is_reliable=True, proportion=1.0), LanguagePrediction(language='zh', probability=0.9999504089355469, is_reliable=True, proportion=1.0), LanguagePrediction(language='ja', probability=1.0, is_reliable=True, proportion=1.0), LanguagePrediction(language='ar', probability=0.9999980926513672, is_reliable=True, proportion=1.0), LanguagePrediction(language='sr', probability=0.3987821042537689, is_reliable=False, proportion=1.0)]


In [9]:
# Check the cluster
from dask.distributed import Client
import joblib
client = Client("3.15.27.177:8786")
client

Client Scheduler: tcp://3.15.27.177:8786 Dashboard: http://3.15.27.177:8787/status,Cluster Workers: 12 Cores: 12 Memory: 50.31 GB


In [10]:
import pandas as pd
from os import listdir
import re, joblib, gzip, gc, warnings
from pandas.io.json import json_normalize
from joblib import Parallel, delayed
warnings.filterwarnings('ignore')

In [11]:
msg_root='./data/channel_msg/channel/'
# list all 2019 files
daily_files=[s+'/' for s in listdir(msg_root) ]
daily_files

['df_telegram_hk_1050512740.pkl.gz/',
 'df_telegram_hk_1119394801.pkl.gz/',
 'df_telegram_hk_1077688074.pkl.gz/',
 'df_telegram_hk_1066791689.pkl.gz/',
 'df_telegram_hk_1038976893.pkl.gz/',
 'df_telegram_hk_1107212914.pkl.gz/',
 'df_telegram_hk_1122313490.pkl.gz/',
 'df_telegram_hk_1121301966.pkl.gz/',
 'df_telegram_hk_1104028405.pkl.gz/',
 'df_telegram_hk_1022574258.pkl.gz/',
 'df_telegram_hk_1119983390.pkl.gz/',
 'df_telegram_hk_1051283327.pkl.gz/',
 'df_telegram_hk_1099491088.pkl.gz/',
 'df_telegram_hk_1138151253.pkl.gz/',
 'df_telegram_hk_1088756981.pkl.gz/',
 'df_telegram_hk_1035312283.pkl.gz/',
 'df_telegram_hk_1123257091.pkl.gz/',
 'df_telegram_hk_1005640892.pkl.gz/',
 'df_telegram_hk_1056767742.pkl.gz/',
 'df_telegram_hk_1128365706.pkl.gz/',
 'df_telegram_hk_1069452092.pkl.gz/',
 'df_telegram_hk_1106240275.pkl.gz/',
 'df_telegram_hk_1057176757.pkl.gz/',
 'df_telegram_hk_1055638806.pkl.gz/',
 'df_telegram_hk_1127055716.pkl.gz/',
 'df_telegram_hk_1112061004.pkl.gz/',
 'df_telegra

In [12]:
df_test = pd.read_pickle(msg_root+daily_files[0][:-1], compression="gzip")
df_test.head()

,_,id,date,message,out,mentioned,media_unread,silent,post,from_scheduled,...,action._,action.photo._,action.photo.id,action.photo.access_hash,action.photo.file_reference,action.photo.date,action.photo.sizes,action.photo.dc_id,action.photo.has_stickers,action.title
0,Message,88,2019-11-25 11:43:18+00:00,盛康選區500多張問題票突然翻生\n民主派趙寶琴由贏變輸 支持者鼓譟\n\n節錄內文：\n\...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
1,Message,87,2019-11-25 01:10:16+00:00,2019年區議會選舉結果 － 青衣\n\nS21 安灝 － 譚家浚（公民黨）\nS22 偉盈...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
2,Message,86,2019-09-13 09:36:25+00:00,[#交通] 今晚中秋節通宵交通服務\n\n今晚（9月13日晚上-9月14日凌晨）九巴青衣路線...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
3,Message,85,2019-09-01 14:45:04+00:00,機場快綫由機場往九龍站/香港站重開（不停青衣站），機場快綫往機場方向、東涌綫全綫及迪士尼站繼...,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
4,Message,84,2019-09-01 10:34:27+00:00,機場快綫、東涌綫、迪士尼綫，全綫暫停服務,False,False,False,False,True,False,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


In [29]:
# from langdetect import detect
import cld3
# import fasttext

msg_root='./data/channel_msg/channel/'


def language_detect(daily_file):
    file_path = msg_root + daily_file[:-1]
    print(daily_file)
    df_msg = pd.read_pickle(file_path, compression="gzip").fillna("-1 -1 -1")
    try :
        messages = df_msg.message.values

    #     df_re = pd.DataFrame(columns=["message", "langdetect", "fasttext", "cld3"])
        df_re = pd.DataFrame(columns=["message", "cld3"])

        langdetect_re = []
        fasttext_re = []
        cld3_re = []

        for msg in messages:
            try:
    #             langdetect_re.append(detect(str(msg)))
                cld3_re.append(cld3.get_language(str(msg))[0])
    #             fasttext_re.append(fasttext.load_model('lid.176.ftz').predict(str(msg))[0][-1])
            except:
    #             langdetect_re.append("error")
    #             fasttext_re.append("error")
                cld3_re.append("error")
        df_re["message"] = df_msg["message"]
    #     df_re["langdetect"] = langdetect_re
        df_re["cld3"] = cld3_re
    #     df_re["fasttext"] = fasttext_re
        df_re.to_pickle(msg_root+'lang/'+ daily_file[:-1], compression='gzip')
    except:
        pass
    gc.collect()

language_detect(daily_files[5])    
    
        
        
    

df_telegram_hk_1107212914.pkl.gz/


In [30]:
df_lang = pd.read_pickle(msg_root+'lang/'+ daily_files[5][:-1], compression='gzip')
df_lang.head()

,message,cld3
0,真邪恶！,zh
1,史上最齊全紅藍黑抵制名單\nhttps://lih.kg/1790784\n- 分享自 LI...,zh
2,桃園 百分百 意八 Itamomo\nhttps://lih.kg/1663422\n- 分...,zh
3,https://www.facebook.com/353665548360498/posts...,hi
4,楊明撐暴警\n罷食楊明燉店\n\nhttps://t.me/hkpicnic\n【訂閱野餐頻...,zh


In [ ]:
%time _ = Parallel(n_jobs=-1)(delayed(language_detect)(daily_file) for daily_file in daily_files)